In [220]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?i=computers&bbn=429886031&rh=n%3A429886031%2Cp_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&dc&page=2&qid=1698924010&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


['/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2SKG4B/ref=sr_1_47?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-47',
 '/Apple-MacBook-laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHHH4T4/ref=sr_1_35?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-35',
 '/Apple-MacBook-laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHTPQBN/ref=sr_1_42?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-42',
 '/2022-Apple-MacBook-laptop-chip/dp/B0B3BK2K8Y/ref=sr_1_43?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-43',
 '/Apple-MacBook-14-inch-8%E2%80%91core-14%E2%80%91core/dp/B0CJCN9QQY/ref=sr_1_48?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031

In [221]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'Macbook'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2SKG4B/ref=sr_1_47?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-47
2 / 240
/Apple-MacBook-laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHHH4T4/ref=sr_1_35?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-35
3 / 240
/Apple-MacBook-laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHTPQBN/ref=sr_1_42?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-42
4 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BK2K8Y/ref=sr_1_43?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-43
5 / 240
/Apple-MacBook-14-inch-8%E2%80%91core-14%E2%80%91core/dp/B0CJCN9QQY/ref=sr_1_48?qid=1698939694&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A41

42 / 240
/2019-Apple-MacBook-Pro-16/dp/B0BB3BK6SG/ref=sr_1_46?qid=1698939699&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-46
43 / 240
/Apple-Macbook-13-inch-Storage-Starlight/dp/B0CJMQ6F1L/ref=sr_1_37?qid=1698939699&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-37
44 / 240
/Apple-MacBook-Air-Late-2018/dp/B07RSDV1SV/ref=sr_1_45?qid=1698939699&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-45
45 / 240
/Apple-MacBook-13-inch-Storage-Renewed/dp/B0BJ41FV7N/ref=sr_1_38?qid=1698939699&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-38
46 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3C14BMY/ref=sr_1_26?qid=1698939699&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-26
47 / 240
/Ap

83 / 240
/Apple-MacBook-256GB-Space-Renewed/dp/B0BJGTPXXZ/ref=sr_1_30?qid=1698939709&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-30
84 / 240
/Apple-Macbook-13-inch-Storage-Starlight/dp/B0CJMQ6F1L/ref=sr_1_37?qid=1698939709&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-37
85 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BJD4F4/ref=sr_1_27?qid=1698939709&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-27
86 / 240
/Apple-MacBook-14-inch-14%E2%80%91core-512GB/dp/B0B9KG779B/ref=sr_1_34?qid=1698939709&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-34
87 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2SKG4B/ref=sr_1_47?qid=1698939709&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&s

124 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BJD4F4/ref=sr_1_27?qid=1698939718&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-27
125 / 240
/Apple-MacBook-14-inch-14%E2%80%91core-512GB/dp/B0B9KG779B/ref=sr_1_34?qid=1698939718&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-34
126 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BK2K8Y/ref=sr_1_43?qid=1698939718&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-43
127 / 240
/Apple-MacBook-Chip-13-inch-256GB/dp/B08XY2779H/ref=sr_1_33?qid=1698939718&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-33
128 / 240
/Apple-MacBook-laptop-12%E2%80%91core-30%E2%80%91core/dp/B0BSHRZ6DS/ref=sr_1_31?qid=1698939718&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=41915203

164 / 240
/Apple-MacBook-256GB-Space-Renewed/dp/B0BJGTPXXZ/ref=sr_1_30?qid=1698939723&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-30
165 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3C14BMY/ref=sr_1_26?qid=1698939723&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-26
166 / 240
/Apple-MacBook-laptop-12%E2%80%91core-19%E2%80%91core/dp/B0BSHVC2QW/ref=sr_1_32?qid=1698939723&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-32
167 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2WTNQ6/ref=sr_1_25?qid=1698939723&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-25
168 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BJD4F4/ref=sr_1_27?qid=1698939723&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=comput

205 / 240
/Apple-MacBook-14-inch-8%E2%80%91core-14%E2%80%91core/dp/B0CJCN9QQY/ref=sr_1_48?qid=1698939732&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-48
206 / 240
/2022-Apple-MacBook-laptop-chip/dp/B0B3BG5DJ7/ref=sr_1_43?qid=1698939732&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-43
207 / 240
/2019-Apple-MacBook-Pro-16/dp/B0BB3BK6SG/ref=sr_1_46?qid=1698939732&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-46
208 / 240
/Apple-MacBook-14-inch-14%E2%80%91core-512GB/dp/B0B9KG779B/ref=sr_1_35?qid=1698939732&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=computers&sr=1-35
209 / 240
/Apple-MacBook-Chip-13-inch-512GB/dp/B08R2WTNQ6/ref=sr_1_26?qid=1698939732&refinements=p_n_feature_twenty-two_browse-bin%3A27387844031%2Cp_72%3A419156031&rnid=419152031&s=c

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Ear placement,Charging time,Style,Shape,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223,"Smart Watch for Women(Answer/Make Call), Alexa...",44..,4.3 out of 5,"1,601 global ratings",https://www.amazon.co.uk/Fitness-Monitor-Water...,Value not found,Tensky,ID208BT,NaN,NaN,...,NaN,NaN,"Casual, Classic, Hybrid, Minimalist, Modern, R...",Rectangular,"Women, Men, Unisex Adults, Unisex Youth",NaN,NaN,NaN,NaN,NaN
1224,"Smart Watch for Women(Answer/Make Call), Alexa...",44..,4.3 out of 5,"1,601 global ratings",https://www.amazon.co.uk/Fitness-Monitor-Water...,Value not found,Tensky,ID208BT,NaN,NaN,...,NaN,NaN,"Casual, Classic, Hybrid, Minimalist, Modern, R...",Rectangular,"Women, Men, Unisex Adults, Unisex Youth",NaN,NaN,NaN,NaN,NaN
1225,"Smart Watch for Women(Answer/Make Call), Alexa...",44..,4.3 out of 5,"1,601 global ratings",https://www.amazon.co.uk/Fitness-Monitor-Water...,Value not found,Tensky,ID208BT,NaN,NaN,...,NaN,NaN,"Casual, Classic, Hybrid, Minimalist, Modern, R...",Rectangular,"Women, Men, Unisex Adults, Unisex Youth",NaN,NaN,NaN,NaN,NaN
1226,"Smart Watch for Women(Answer/Make Call), Alexa...",44..,4.3 out of 5,"1,601 global ratings",https://www.amazon.co.uk/Fitness-Monitor-Water...,Value not found,Tensky,ID208BT,NaN,NaN,...,NaN,NaN,"Casual, Classic, Hybrid, Minimalist, Modern, R...",Rectangular,"Women, Men, Unisex Adults, Unisex Youth",NaN,NaN,NaN,NaN,NaN


In [222]:
df.Search.value_counts()

iPhone         609
AirPods        255
iPad           168
Macbook        107
Apple Watch     79
Name: Search, dtype: int64

In [223]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=iphone&i=electronics&rh=n%3A560820%2Cp_72%3A419156031&dc&page=2&qid=1698924321&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


['/Apple-iPhone-12-128GB-Blue/dp/B08PCCJP5C/ref=sr_1_25?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-25',
 '/Apple-MNOW2B-SIM-Free-Smartphone-Refurbished-Space-Grey/dp/B075D79LX1/ref=sr_1_47?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-47',
 '/Apple-iPhone-13-128GB-Pink/dp/B09G99YSFG/ref=sr_1_41?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-41',
 '/Apple-iPhone-14-Plus-128/dp/B0BDJY2DFH/ref=sr_1_46?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-46',
 '/Apple-iPhone-13-128-GB/dp/B09V43FPYG/ref=sr_1_33?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-33',
 '/Apple-iPhone-15-Plus-128/dp/B0CHX2DRNW/ref=sr_1_39?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-39',
 '/Apple-iPhone-mini-128GB-Black/dp/B08PCDKD6G/ref=sr_1_38?ke

In [224]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'iPhone'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 240
/Apple-iPhone-12-128GB-Blue/dp/B08PCCJP5C/ref=sr_1_25?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-25
2 / 240
/Apple-MNOW2B-SIM-Free-Smartphone-Refurbished-Space-Grey/dp/B075D79LX1/ref=sr_1_47?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-47
3 / 240
/Apple-iPhone-13-128GB-Pink/dp/B09G99YSFG/ref=sr_1_41?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-41
4 / 240
/Apple-iPhone-14-Plus-128/dp/B0BDJY2DFH/ref=sr_1_46?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-46
5 / 240
/Apple-iPhone-13-128-GB/dp/B09V43FPYG/ref=sr_1_33?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-33
6 / 240
/Apple-iPhone-15-Plus-128/dp/B0CHX2DRNW/ref=sr_1_39?keywords=iphone&qid=1698941374&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-39
7 / 240
/Apple-iPhone-mini-128GB-Blac

52 / 240
/Apple-iPhone-mini-64GB-Black/dp/B08PCF2P4R/ref=sr_1_35?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-35
53 / 240
/Apple-iPhone-mini-128GB-Black/dp/B08PCDKD6G/ref=sr_1_38?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-38
54 / 240
/Apple-iPhone-15-Plus-128/dp/B0CHX2DRNW/ref=sr_1_39?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-39
55 / 240
/Apple-iPhone-13-128GB-Pink/dp/B09G99YSFG/ref=sr_1_41?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-41
56 / 240
/Apple-iPhone-14-128-GB/dp/B0BDJJKPT2/ref=sr_1_34?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-34
57 / 240
/Apple-iPhone-12-64GB-White/dp/B08PCG664W/ref=sr_1_26?keywords=iphone&qid=1698941385&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-26
58 / 240
/Apple-iPhone-15-Pro-256/dp/B0CHX5R7K3/ref=s

103 / 240
/Apple-iPhone-12-128GB-Black/dp/B08PCGL5TH/ref=sr_1_48?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-48
104 / 240
/Apple-iPhone-13-128-GB/dp/B09V43FPYG/ref=sr_1_33?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-33
105 / 240
/Apple-iPhone-12-64GB-White/dp/B08PCG664W/ref=sr_1_26?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-26
106 / 240
/Apple-iPhone-15-Pro-256/dp/B0CHX5R7K3/ref=sr_1_32?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-32
107 / 240
/Apple-iPhone-12-128GB-Blue/dp/B08PCCJP5C/ref=sr_1_25?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-25
108 / 240
/Apple-iPhone-15-Plus-128/dp/B0CHX2DRNW/ref=sr_1_40?keywords=iphone&qid=1698941396&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-40
109 / 240
/Apple-iPhone-128GB-Pacific-Blue/dp/B08PCB8Q

155 / 240
/Apple-iPhone-64GB-Red-Renewed/dp/B07P5NGFR7/ref=sr_1_37?keywords=iphone&qid=1698941407&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-37
156 / 240
/Apple-iPhone-13-128-GB/dp/B09V43FPYG/ref=sr_1_33?keywords=iphone&qid=1698941407&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-33
157 / 240
/Apple-iPhone-14-128GB-Blue/dp/B0BNLXTK3P/ref=sr_1_28?keywords=iphone&qid=1698941407&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-28
158 / 240
/Apple-iPhone-mini-128GB-Blue/dp/B08PCDNC9W/ref=sr_1_45?keywords=iphone&qid=1698941407&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-45
159 / 240
/Apple-iPhone-12-128GB-Blue/dp/B08PCCJP5C/ref=sr_1_25?keywords=iphone&qid=1698941407&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-25
160 / 240
/Apple-iPhone-12-64GB-White/dp/B08PCG664W/ref=sr_1_26?keywords=iphone&qid=1698941407&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-26
161 / 240
/Apple-iPhone-15-128-GB/dp/B0CHX3RP

206 / 240
/Apple-iPhone-15-Plus-128/dp/B0CHX2DRNW/ref=sr_1_39?keywords=iphone&qid=1698941418&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-39
207 / 240
/Apple-iPhone-14-128-GB/dp/B0BDJJKPT2/ref=sr_1_34?keywords=iphone&qid=1698941418&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-34
208 / 240
/Apple-iPhone-12-64GB-White/dp/B08PCG664W/ref=sr_1_26?keywords=iphone&qid=1698941418&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-26
209 / 240
/Apple-iPhone-14-Plus-128/dp/B0BDJY2DFH/ref=sr_1_46?keywords=iphone&qid=1698941418&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-46
210 / 240
/Apple-iPhone-12-64GB-Renewed-Purple/dp/B0B42V45SV/ref=sr_1_30?keywords=iphone&qid=1698941418&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-30
211 / 240
/Apple-iPhone-Pro-128GB-Silver/dp/B08PCCM1GC/ref=sr_1_31?keywords=iphone&qid=1698941418&refinements=p_72%3A419156031&rnid=419152031&s=telephone&sr=1-31
212 / 240
/Apple-iPhone-mini-128GB-Black/d

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Ear placement,Charging time,Style,Shape,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,"Apple iPhone 14, 128GB, Blue (Renewed)",550..,4.5 out of 5,64 global ratings,https://www.amazon.co.uk/Apple-iPhone-14-128GB...,Value not found,Apple,IPhone 14,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1235,"Apple iPhone 14, 128GB, Blue (Renewed)",550..,4.5 out of 5,64 global ratings,https://www.amazon.co.uk/Apple-iPhone-14-128GB...,Value not found,Apple,IPhone 14,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1236,"Apple iPhone 14, 128GB, Blue (Renewed)",550..,4.5 out of 5,64 global ratings,https://www.amazon.co.uk/Apple-iPhone-14-128GB...,Value not found,Apple,IPhone 14,IOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1237,"Apple iPhone 14, 128GB, Blue (Renewed)",550..,4.5 out of 5,64 global ratings,https://www.amazon.co.uk/Apple-iPhone-14-128GB...,Value not found,Apple,iPhone 14,iOS 16,5G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
df.Search.value_counts()

iPhone         620
AirPods        255
iPad           168
Macbook        107
Apple Watch     79
Name: Search, dtype: int64

In [226]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=ipad&i=electronics&page=2&crid=2SLN76UU27TLB&qid=1698924622&sprefix=ipad%2Celectronics%2C98&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


['/2022-Apple-10-9-inch-iPad-Air-Wi-Fi/dp/B09V4K8868/ref=sr_1_37?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-37',
 '/Apple-iPad-128GB-Wi-Fi-Cellular/dp/B08N89WSZ7/ref=sr_1_28?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-28',
 '/2021-Apple-10-2-inch-Wi-Fi-256GB/dp/B09N6LHQSB/ref=sr_1_39?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-39',
 '/Apple-Generation-128GB-Retail-Renewed/dp/B084H2HDND/ref=sr_1_30?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-30',
 '/Apple-iPad-10-2-32GB-Wi-Fi/dp/B08N89P2QZ/ref=sr_1_41?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-41',
 '/Apple-iPad-2017-32GB-Wi-Fi/dp/B07HZ3MB19/ref=sr_1_43?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-43',
 '/2022-A

In [227]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'iPad'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 242
/2022-Apple-10-9-inch-iPad-Air-Wi-Fi/dp/B09V4K8868/ref=sr_1_37?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-37
2 / 242
/Apple-iPad-128GB-Wi-Fi-Cellular/dp/B08N89WSZ7/ref=sr_1_28?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-28
3 / 242
/2021-Apple-10-2-inch-Wi-Fi-256GB/dp/B09N6LHQSB/ref=sr_1_39?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-39
4 / 242
/Apple-Generation-128GB-Retail-Renewed/dp/B084H2HDND/ref=sr_1_30?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-30
5 / 242
/Apple-iPad-10-2-32GB-Wi-Fi/dp/B08N89P2QZ/ref=sr_1_41?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-41
6 / 242
/Apple-iPad-2017-32GB-Wi-Fi/dp/B07HZ3MB19/ref=sr_1_43?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943200&s=electronics&sprefix=ipad%2Celectronic

49 / 242
/Apple-iPad-Mini-Space-Refurbished-Grey/dp/B01LZ2GACT/ref=sr_1_45?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943210&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-45
50 / 242
/Apple-iPad-12-9-inch-Wi-Fi-256GB/dp/B086M8CGRT/ref=sr_1_42?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943210&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-42
51 / 242
/Apple-iPad-10-2-32GB-4G/dp/B0845WKG2P/ref=sr_1_34?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943210&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-34
52 / 242
/Apple-iPad-Air-16GB-Wi-Fi/dp/B00LJPHPGW/ref=sr_1_38?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943210&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-38
53 / 242
/Apple-iPad-10-2-inch-Wi-Fi-Cellular/dp/B08N8D3XHB/ref=sr_1_35?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943210&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-35
54 / 242
/Apple-MF532LL-Verizon-VERSION-Refurbished/dp/B00I8LAB7U/ref=sr_1_47?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943210&s=electronics&sprefix=ipad

97 / 242
/Apple-Generation-128GB-Retail-Renewed/dp/B084H2HDND/ref=sr_1_30?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943220&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-30
98 / 242
/Apple-2022-10-9-inch-iPad-Wi-Fi/dp/B0BJMDHJWG/ref=sr_1_29?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943220&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-29
99 / 242
/2022-Apple-10-9-inch-iPad-Air-Wi-Fi/dp/B09V4K8868/ref=sr_1_37?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943220&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-37
100 / 242
/Apple-iPad-Air-16GB-Silver/dp/B077J7H62P/ref=sr_1_31?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943220&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-31
101 / 242
/Apple-iPad-WI-FI-Space-Renewed/dp/B0855NVMVR/ref=sr_1_27?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943220&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-27
102 / 242
/Bluetooth-Split-Screen-Function-Headphone-Keyboard/dp/B0CDXHS4H6/ref=sr_1_48?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943220&s=electr

145 / 242
/Apple-MF532LL-Verizon-VERSION-Refurbished/dp/B00I8LAB7U/ref=sr_1_47?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943229&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-47
146 / 242
/Apple-iPad-128GB-Wi-Fi-Cellular/dp/B08N89WSZ7/ref=sr_1_28?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943229&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-28
147 / 242
/Apple-2022-10-9-inch-iPad-Wi-Fi/dp/B0BJMDHJWG/ref=sr_1_29?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943229&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-29
148 / 242
/Apple-iPad-10-2-128GB-Wi-Fi/dp/B08N8L8W21/ref=sr_1_40?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943229&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-40
149 / 242
/Apple-iPad-Air-16GB-Silver/dp/B077J7H62P/ref=sr_1_31?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943229&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-31
150 / 242
/Apple-iPad-WI-FI-Space-Renewed/dp/B0855NVMVR/ref=sr_1_27?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943229&s=electronics&sprefix=ipad%2

193 / 242
/Samsung-Android-Version-Manufacturer-Warranty/dp/B0BD4XHGQN/ref=sr_1_48?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943238&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-48
194 / 242
/Apple-iPad-16GB-Wi-Fi-Refurbished/dp/B074JFBNC6/ref=sr_1_27?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943238&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-27
195 / 242
/Apple-iPad-10-2-128GB-Wi-Fi/dp/B08N8L8W21/ref=sr_1_42?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943238&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-42
196 / 242
/Apple-iPad-Air-16GB-Wi-Fi/dp/B00LJPHPGW/ref=sr_1_40?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943238&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-40
197 / 242
/Apple-2022-10-9-inch-iPad-Wi-Fi/dp/B0BJMDHJWG/ref=sr_1_31?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943238&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-31
198 / 242
/2022-Apple-10-9-inch-iPad-Air-Wi-Fi/dp/B09V4K8868/ref=sr_1_39?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943238&s=electronics&spre

238 / 242
/Apple-iPad-10-2-inch-Wi-Fi-Cellular/dp/B08N8D3XHB/ref=sr_1_35?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943243&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-35
239 / 242
/2021-Apple-10-2-inch-Wi-Fi-Cellular/dp/B09G9DLBR8/ref=sr_1_33?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943243&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-33
240 / 242
/Apple-iPad-2017-32GB-Wi-Fi/dp/B07HZ3MB19/ref=sr_1_43?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943243&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-43
241 / 242
/iPad-Air-16GB-WiFi-Cellular/dp/B09LD1D91V/ref=sr_1_32?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943243&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-32
242 / 242
/Samsung-Android-Version-Manufacturer-Warranty/dp/B0BD4XHGQN/ref=sr_1_46?crid=2SLN76UU27TLB&keywords=ipad&qid=1698943243&s=electronics&sprefix=ipad%2Celectronics%2C98&sr=1-46


,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product,Number of batteries,Are batteries included,Batteries required?,Battery description
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,"【2023 Newest】iPad Pencil (2nd Generation), iSk...",23..,4.4 out of 5,342 global ratings,https://www.amazon.co.uk/Newest%E3%80%91iPad-G...,Value not found,ISkey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1 Lithium Polymer batteries required. (included),Yes,Yes,Lithium-Ion
1269,"【2023 Newest】iPad Pencil (2nd Generation), iSk...",23..,4.4 out of 5,342 global ratings,https://www.amazon.co.uk/Newest%E3%80%91iPad-G...,Value not found,ISkey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1 Lithium Polymer batteries required. (included),Yes,Yes,Lithium-Ion
1270,"【2023 Newest】iPad Pencil (2nd Generation), iSk...",23..,4.4 out of 5,342 global ratings,https://www.amazon.co.uk/Newest%E3%80%91iPad-G...,Value not found,ISkey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1 Lithium Polymer batteries required. (included),Yes,Yes,Lithium-Ion
1271,"【2023 Newest】iPad Pencil (2nd Generation), iSk...",23..,4.4 out of 5,342 global ratings,https://www.amazon.co.uk/Newest%E3%80%91iPad-G...,Value not found,ISkey,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1 Lithium Polymer batteries required. (included),Yes,Yes,Lithium-Ion


In [228]:
df.Search.value_counts()

iPhone         620
AirPods        255
iPad           202
Macbook        107
Apple Watch     79
Name: Search, dtype: int64

In [229]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=AirPods&i=electronics&rh=n%3A4085731%2Cp_72%3A419156031&dc&page=2&qid=1698924837&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


['/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-rose-gold/dp/B0BD4Z526Y/ref=sr_1_42?keywords=AirPods&qid=1698944894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-42',
 '/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Blue/dp/B0CBX76S6G/ref=sr_1_45?keywords=AirPods&qid=1698944894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-45',
 '/Wireless-Headphones-Bluetooth-Cancellation-Waterproof-Pink/dp/B0BGMDVM7V/ref=sr_1_40?keywords=AirPods&qid=1698944894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-40',
 '/JBL-Tune-230NC-Ear-Headphones-Black/dp/B09HGR1DVC/ref=sr_1_29?keywords=AirPods&qid=1698944894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-29',
 '/Wireless-Bluetooth-Headphones-Earphones-Waterproof-Black/dp/B0C2KDYX4Q/ref=sr_1_32?keywords=AirPods&qid=1698944894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-32',
 '/Headphones-Bluetooth-Cancellation-Waterproof-Earphones-Black/dp/B0BT87ZYJG/ref=

In [230]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'AirPods'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 242
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-rose-gold/dp/B0BD4Z526Y/ref=sr_1_42?keywords=AirPods&qid=1698944894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-42
2 / 242
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Blue/dp/B0CBX76S6G/ref=sr_1_45?keywords=AirPods&qid=1698944894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-45
3 / 242
/Wireless-Headphones-Bluetooth-Cancellation-Waterproof-Pink/dp/B0BGMDVM7V/ref=sr_1_40?keywords=AirPods&qid=1698944894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-40
4 / 242
/JBL-Tune-230NC-Ear-Headphones-Black/dp/B09HGR1DVC/ref=sr_1_29?keywords=AirPods&qid=1698944894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-29
5 / 242
/Wireless-Bluetooth-Headphones-Earphones-Waterproof-Black/dp/B0C2KDYX4Q/ref=sr_1_32?keywords=AirPods&qid=1698944894&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-32
6 / 242
/Headphones-Bluetooth-Cancellation-Waterproof-Earphone

45 / 242
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Black/dp/B09BZ64R7S/ref=sr_1_34?keywords=AirPods&qid=1698944899&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-34
46 / 242
/Bluetooth-Headphones-Waterproof-Immersive-Cancelling-white/dp/B0C7JVV8G8/ref=sr_1_28?keywords=AirPods&qid=1698944899&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-28
47 / 242
/Bluetooth-Headphones-Canceling-Waterproof-Earphones-White/dp/B0BWY2HSVT/ref=sr_1_46?keywords=AirPods&qid=1698944899&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-46
48 / 242
/Wireless-Headphones-Bluetooth-Cancellation-Waterproof-Pink/dp/B0BGMDVM7V/ref=sr_1_40?keywords=AirPods&qid=1698944899&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-40
49 / 242
/Bluetooth-Headphones-Cancelling-Earphones-Waterproof-White/dp/B0BYD8F9W7/ref=sr_1_27?keywords=AirPods&qid=1698944904&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-27
50 / 242
/Bluetooth-Headphones-Cance

89 / 242
/JBL-Tune-230NC-Ear-Headphones-Black/dp/B09HGR1DVC/ref=sr_1_29?keywords=AirPods&qid=1698944909&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-29
90 / 242
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Black/dp/B09BZ64R7S/ref=sr_1_35?keywords=AirPods&qid=1698944909&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-35
91 / 242
/Wireless-Earbuds-Bluetooth-Headphones-Waterproof-White/dp/B08J7B4GMC/ref=sr_1_38?keywords=AirPods&qid=1698944909&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-38
92 / 242
/Beats-Studio-Buds-Cancelling-Built/dp/B0979QRKMW/ref=sr_1_26?keywords=AirPods&qid=1698944909&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-26
93 / 242
/STUDIO-NOD-Bluetooth-detection-microphone-Grey/dp/B0BRTZ2QSX/ref=sr_1_46?keywords=AirPods&qid=1698944909&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-46
94 / 242
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Black/dp/B0B824NB2D/ref=sr_1_45?key

133 / 242
/JBL-Tune-230NC-Ear-Headphones-Black/dp/B09HGR1DVC/ref=sr_1_29?keywords=AirPods&qid=1698944919&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-29
134 / 242
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Black/dp/B0BHWCJNWP/ref=sr_1_44?keywords=AirPods&qid=1698944919&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-44
135 / 242
/Certified-Light-Weight-Headphones-Microphone-Cancelling/dp/B0BPX31YB3/ref=sr_1_30?keywords=AirPods&qid=1698944919&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-30
136 / 242
/Bluetooth-Headphones-Waterproof-Immersive-Cancelling-white/dp/B0C7JVV8G8/ref=sr_1_27?keywords=AirPods&qid=1698944919&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-27
137 / 242
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Black/dp/B09BZ64R7S/ref=sr_1_35?keywords=AirPods&qid=1698944919&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-35
138 / 242
/Bluetooth-Headphones-Canceling-Waterproof-Ear

177 / 242
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Black/dp/B0BHWCJNWP/ref=sr_1_44?keywords=AirPods&qid=1698944928&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-44
178 / 242
/Wireless-Bluetooth-Headphones-Earphones-Waterproof-Black/dp/B0C2KDYX4Q/ref=sr_1_32?keywords=AirPods&qid=1698944928&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-32
179 / 242
/Wireless-Bluetooth-Earphones-Charging-Playback-White/dp/B0C9JS28V6/ref=sr_1_39?keywords=AirPods&qid=1698944928&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-39
180 / 242
/Beats-Studio-Buds-Cancelling-Built/dp/B0979QRKMW/ref=sr_1_25?keywords=AirPods&qid=1698944928&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-25
181 / 242
/Wireless-Headphones-Bluetooth-Cancellation-Waterproof-Pink/dp/B0BGMDVM7V/ref=sr_1_40?keywords=AirPods&qid=1698944928&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-40
182 / 242
/Certified-Light-Weight-Headphones-Microphone-Cancel

219 / 242
/Sony-WF-C500-Wireless-Headphones-Built-Black/dp/B09FKGJ1CB/ref=sr_1_37?keywords=AirPods&qid=1698944938&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-37
220 / 242
/JBL-Wave-200TWS-Wireless-Headphones-Black/dp/B09HGGV5R5/ref=sr_1_34?keywords=AirPods&qid=1698944938&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-34
221 / 242
/Bluetooth-Headphones-Waterproof-Immersive-Cancelling-white/dp/B0C7JVV8G8/ref=sr_1_27?keywords=AirPods&qid=1698944938&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-27
222 / 242
/Bluetooth-Headphones-Earphones-Cancelling-Waterproof-Black/dp/B0B824NB2D/ref=sr_1_45?keywords=AirPods&qid=1698944938&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-45
223 / 242
/Beats-Fit-Pro-Cancelling-Built/dp/B09Q768ND9/ref=sr_1_33?keywords=AirPods&qid=1698944938&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-33
224 / 242
/Bluetooth-Headphones-Canceling-Waterproof-Earphones-White/dp/B0BWY2HSVT/ref=sr

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product,Number of batteries,Are batteries included,Batteries required?,Battery description
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,"Wireless Earbuds Bluetooth, Wireless Bluetooth...",12..,4.8 out of 5,"7,958 global ratings",https://www.amazon.co.uk/Wireless-Bluetooth-He...,Value not found,Aimoh,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1339,"Wireless Earbuds Bluetooth, Wireless Bluetooth...",12..,4.8 out of 5,"7,958 global ratings",https://www.amazon.co.uk/Wireless-Bluetooth-He...,Value not found,Aimoh,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1340,LEYUSMART Bluetooth Earbuds for iPhone 14 /Pro...,11..,3.8 out of 5,76 global ratings,https://www.amazon.co.uk/Leyu-Smart-Bluetooth-...,Value not found,Leyu Smart,W-B-Earbud-001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1341,JBL Wave 200TWS Wireless In-Ear Headphones - B...,42..,4.1 out of 5,"13,448 global ratings",https://www.amazon.co.uk/JBL-Wave-200TWS-Wirel...,Value not found,JBL,JBLW200TWSBLK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [231]:
df.Search.value_counts()

iPhone         620
AirPods        325
iPad           202
Macbook        107
Apple Watch     79
Name: Search, dtype: int64

In [232]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(10):
    url = 'https://www.amazon.co.uk/s?k=apple+watch&i=electronics&rh=n%3A560798%2Cp_72%3A419156031&dc&page=2&qid=1698925012&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


['/Apple-Watch-generation-Smart-watch/dp/B0BDHW118S/ref=sr_1_25?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-25',
 '/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX33TL3/ref=sr_1_27?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-27',
 '/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX1DDVW/ref=sr_1_29?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-29',
 '/Apple-Watch-generation-Smart-watch/dp/B0BDJ7S56L/ref=sr_1_32?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-32',
 '/Apple-Watch-generation-Smart-watch/dp/B0BDHYDYFP/ref=sr_1_38?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-38',
 '/Apple-Watch-45mm-Smart-watch/dp/B0BDHQYB3Q/ref=sr_1_31?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=el

In [233]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        #context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        context = await browser.new_context(user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    try:
        temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    except:
        continue
    temp_df['Search'] = 'Apple Watch'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 240
/Apple-Watch-generation-Smart-watch/dp/B0BDHW118S/ref=sr_1_25?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-25
2 / 240
/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX33TL3/ref=sr_1_27?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-27
3 / 240
/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX1DDVW/ref=sr_1_29?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-29
4 / 240
/Apple-Watch-generation-Smart-watch/dp/B0BDJ7S56L/ref=sr_1_32?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-32
5 / 240
/Apple-Watch-generation-Smart-watch/dp/B0BDHYDYFP/ref=sr_1_38?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-38
6 / 240
/Apple-Watch-45mm-Smart-watch/dp/B0BDHQYB3Q/ref=sr_1_31?keywords=apple+watch&qid=1698946567&refinements=p_72%3A419

47 / 240
/Apple-Watch-generation-Smart-watch/dp/B0BDHW118S/ref=sr_1_25?keywords=apple+watch&qid=1698946572&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-25
48 / 240
/Apple-Watch-generation-Smart-watch/dp/B0BDJ7S56L/ref=sr_1_32?keywords=apple+watch&qid=1698946572&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-32
49 / 240
/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX1DDVW/ref=sr_1_30?keywords=apple+watch&qid=1698946576&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-30
50 / 240
/Apple-Smartwatch-Starlight-Aluminium-Detection/dp/B0CHWVB6PJ/ref=sr_1_27?keywords=apple+watch&qid=1698946576&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-27
51 / 240
/Apple-Watch-40mm-GPS-Aluminium/dp/B08QJQVXG5/ref=sr_1_37?keywords=apple+watch&qid=1698946576&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-37
52 / 240
/Apple-Watch-38mm-Aluminum-Sport-Gold-Case-Pink-Sand-Band-GPS-Only/dp/B07CBJBDBC/ref=sr_1_44?keywords=apple+w

93 / 240
/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX1DDVW/ref=sr_1_30?keywords=apple+watch&qid=1698946581&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-30
94 / 240
/Apple-Watch-generation-Smart-watch/dp/B0BDHYDYFP/ref=sr_1_39?keywords=apple+watch&qid=1698946581&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-39
95 / 240
/Apple-Smartwatch-Aluminium-Fitness-Detection/dp/B0CHWV5J6C/ref=sr_1_47?keywords=apple+watch&qid=1698946581&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-47
96 / 240
/Fitness-Monitor-Waterproof-Counter-Smartwatch-Pink/dp/B0BR8YTL7D/ref=sr_1_38?keywords=apple+watch&qid=1698946581&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-38
97 / 240
/Apple-Cellular-Smartwatch-Precision-Extra-Long/dp/B0CHWV2XDP/ref=sr_1_43?keywords=apple+watch&qid=1698946585&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-43
98 / 240
/Apple-Smartwatch-Starlight-Aluminium-Detection/dp/B0CHWVB6PJ/ref=sr_1_31?keywo

139 / 240
/Apple-Smartwatch-Midnight-Aluminium-Detection/dp/B0CHWV6HBV/ref=sr_1_36?keywords=apple+watch&qid=1698946590&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-36
140 / 240
/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX1DDVW/ref=sr_1_29?keywords=apple+watch&qid=1698946590&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-29
141 / 240
/Apple-Watch-40mm-GPS-Aluminium/dp/B08QCSHB3V/ref=sr_1_40?keywords=apple+watch&qid=1698946590&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-40
142 / 240
/Apple-Smartwatch-Aluminium-Fitness-Detection/dp/B0CHWV5J6C/ref=sr_1_47?keywords=apple+watch&qid=1698946590&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-47
143 / 240
/Apple-Smartwatch-Starlight-Aluminum-Resistant/dp/B0CHWV6H2X/ref=sr_1_26?keywords=apple+watch&qid=1698946590&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-26
144 / 240
/Apple-Watch-41mm-Smart-watch/dp/B0BDJG83L4/ref=sr_1_39?keywords=apple+watch&qid=16

185 / 240
/Apple-Smartwatch-Starlight-Aluminum-Resistant/dp/B0CHWV6H2N/ref=sr_1_35?keywords=apple+watch&qid=1698946600&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-35
186 / 240
/Apple-Watch-38mm-Aluminum-Sport-Gold-Case-Pink-Sand-Band-GPS-Only/dp/B07CBJBDBC/ref=sr_1_45?keywords=apple+watch&qid=1698946600&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-45
187 / 240
/Apple-Watch-generation-Smart-watch/dp/B0BDHYDYFP/ref=sr_1_38?keywords=apple+watch&qid=1698946600&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-38
188 / 240
/Apple-Cellular-Smartwatch-Precision-Extra-Long/dp/B0CHX1D3BC/ref=sr_1_34?keywords=apple+watch&qid=1698946600&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-34
189 / 240
/Apple-Watch-generation-Smart-watch/dp/B0BDJ3T7MW/ref=sr_1_44?keywords=apple+watch&qid=1698946600&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-44
190 / 240
/Apple-Smartwatch-Aluminium-Fitness-Detection/dp/B0CHX9GDQ1/ref=sr

231 / 240
/Apple-Cellular-Smartwatch-Midnight-Resistant/dp/B0CHX2FKKS/ref=sr_1_48?keywords=apple+watch&qid=1698946609&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-48
232 / 240
/Apple-Smartwatch-Starlight-Aluminium-Detection/dp/B0CHWVB6PJ/ref=sr_1_31?keywords=apple+watch&qid=1698946609&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-31
233 / 240
/Apple-Smartwatch-Starlight-Aluminum-Resistant/dp/B0CHWV6H2N/ref=sr_1_35?keywords=apple+watch&qid=1698946609&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-35
234 / 240
/Apple-Smartwatch-Aluminum-Fitness-Resistant/dp/B0CHX33TL3/ref=sr_1_27?keywords=apple+watch&qid=1698946609&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-27
235 / 240
/Apple-Cellular-Smartwatch-Starlight-Resistant/dp/B0CHWZYB58/ref=sr_1_33?keywords=apple+watch&qid=1698946609&refinements=p_72%3A419156031&rnid=419152031&s=electronics&sr=1-33
236 / 240
/Apple-Watch-40mm-GPS-Aluminium/dp/B08QJQVXG5/ref=sr_1_37?keywords=

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Target audience,Band material type,Display type,Band colour,Case material type,Specific uses for product,Number of batteries,Are batteries included,Batteries required?,Battery description
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340,LEYUSMART Bluetooth Earbuds for iPhone 14 /Pro...,11..,3.8 out of 5,76 global ratings,https://www.amazon.co.uk/Leyu-Smart-Bluetooth-...,Value not found,Leyu Smart,W-B-Earbud-001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1341,JBL Wave 200TWS Wireless In-Ear Headphones - B...,42..,4.1 out of 5,"13,448 global ratings",https://www.amazon.co.uk/JBL-Wave-200TWS-Wirel...,Value not found,JBL,JBLW200TWSBLK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1342,LEYUSMART Bluetooth Earbuds for iPhone 14 /Pro...,11..,3.8 out of 5,76 global ratings,https://www.amazon.co.uk/Leyu-Smart-Bluetooth-...,Value not found,Leyu Smart,W-B-Earbud-001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1343,"Smart Watch for Women(Answer/Make Call), Alexa...",44..,4.3 out of 5,"1,601 global ratings",https://www.amazon.co.uk/Fitness-Monitor-Water...,Value not found,Tensky,ID208BT,NaN,NaN,...,"Women, Men, Unisex Adults, Unisex Youth",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [234]:
df.Search.value_counts()

iPhone         620
AirPods        325
iPad           202
Macbook        107
Apple Watch     81
Name: Search, dtype: int64

In [ ]:
Mac
iPhone
iPad
AirPods
Watch


In [175]:
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

class AmazonScraper:
    def __init__(
        self,
        headless=True,
        load_images=False,
        chromedriver_path="./chromedriver",
        window_size=(700,900),
        ):
        options = webdriver.ChromeOptions()
        
        if headless:
            options.add_argument("--headless")
            options.add_argument("--disable-gpu")
        
        if not load_images:
            prefs = {"profile.managed_default_content_settings.images": 2}
            options.add_experimental_option("prefs", prefs)
        
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        self.driver = webdriver.Chrome(
            executable_path=chromedriver_path, options=options
        )
        self.driver.set_window_size(*window_size)
            

In [180]:
scraper = AmazonScraper(headless=False)

In [188]:
scraper.driver.get('https://www.amazon.co.uk/Apple-iPhone-11-64GB-White/dp/B08L6XKFRY/ref=cm_cr_arp_d_product_top?ie=UTF8'
                  )

In [192]:
scraper.driver.find_element_by_class_name('a-size-base review-text').text


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".a-size-base review-text"}
  (Session info: chrome=119.0.6045.105)
